In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import math
from sklearn.preprocessing import LabelEncoder

In [2]:
# importing datasets
cal= pd.read_csv("calendar.csv")
sales= pd.read_csv("sales_train_evaluation.csv")
price= pd.read_csv("sell_prices.csv")

In [3]:
#filling all null values with None, null values are only in events features
cal.fillna('None', inplace= True)

In [4]:
#creating data from d_1942 to d_1969 for prediciton
uid= list(sales['id'].unique())
d_cols_after_1941= ['d_'+str(i) for i in range(1942, 1942+28)]

#sales after 1941
sales_after_1941= pd.DataFrame(columns= d_cols_after_1941)
sales_after_1941['id']= uid
sales_after_1941= sales_after_1941[['id']+ d_cols_after_1941]

sales_df= sales.merge(sales_after_1941, on='id')
sales_df.shape

(30490, 1975)

In [5]:
del sales, sales_after_1941
gc.collect()

20

In [6]:
def downcasting_datatype(data):
    '''this function downcasts the size of the DataFrame so as to save memory from crashing'''
    for cols in data.columns:
        if data[cols].dtype == 'float64':
            data[cols] = data[cols].astype(np.float32)
        elif data[cols].dtype in ['int64', 'int32']:
            data[cols]= data[cols].astype(np.int16)
    return data
    
#calculating the size of the dataframes before downcasting
cal_size_before= np.round(cal.memory_usage().sum()/1024,1)
sales_size_before= np.round(sales_df.memory_usage().sum()/(1024*1024),1)
price_size_before= np.round(price.memory_usage().sum()/(1024*1024),1)

'''function calls'''
cal= downcasting_datatype(cal)
sales_df= downcasting_datatype(sales_df)
price= downcasting_datatype(price)

#printing the size before downcasting the size of the data
print("Before downcasting the sizes of the dataframes are:" '\n', 
      "cal:",str(cal_size_before)+"+ KB",'\n'  
      'sales:',str(sales_size_before)+"+ MB", '\n' 
      "price:",str(price_size_before)+"+ MB", "\n")

#calculating the size of the dataframes after downcasting
cal_size_after= np.round(cal.memory_usage().sum()/1024,1)
sales_size_after= np.round(sales_df.memory_usage().sum()/(1024*1024),1)
price_size_after= np.round(price.memory_usage().sum()/(1024*1024),1)

#printing the size after downcasting the size of the data
print("after downcasting the sizes of the dataframes are:" '\n', 
      "cal:",str(cal_size_after)+"+ KB",'\n'  
      'sales:',str(sales_size_after)+"+ MB", '\n' 
      "price:",str(price_size_after)+"+ MB", "\n")

#calculating the percentage reduction of the shape in the data
cal_size_reduction_percentage= np.round((cal_size_after/ cal_size_before) * 100,2)  
sales_size_reduction_percentage= np.round((sales_size_after/ sales_size_before) * 100 ,2) 
sales_size_reduction_percentage= np.round((price_size_after/ price_size_before) * 100 , 2) 

#printing the percentage reduction of the shape in the data
print("percentage reduction in the size of DataFrames:", "\n"
     "cal:", str(cal_size_reduction_percentage)+"% reduction","\n"
     "sales:", str(sales_size_reduction_percentage)+'% reduction',"\n"
     "price:", str(sales_size_reduction_percentage)+"% reduction")

Before downcasting the sizes of the dataframes are:
 cal: 215.5+ KB 
sales: 459.7+ MB 
price: 208.8+ MB 

after downcasting the sizes of the dataframes are:
 cal: 134.7+ KB 
sales: 121.0+ MB 
price: 143.5+ MB 

percentage reduction in the size of DataFrames: 
cal: 62.51% reduction 
sales: 68.73% reduction 
price: 68.73% reduction


In [8]:
#filling all null values with 0.
sales_df.fillna(0,inplace= True)

In [9]:
d_cols = sales_df.columns.tolist()[6:] # Sales columns
# non_d_cols= sales.columns[:6]
non_d_cols = list(reversed(sales_df.columns.tolist()[:6])) 

# Transform Data Structure
data = sales_df.set_index(non_d_cols)
# the following will make one column for sales and one columns for "d" values (d_1 ... d_1913)
data = data.stack()
data = data.to_frame() 
data.columns = ["sales"]
data.reset_index(inplace=True)
data.columns = non_d_cols + ["d", "sales"]

#combining calander and sell_price datataframe twith data to create one single dataframe
data = pd.merge(data, cal, on='d', how='left')
data = pd.merge(data, price, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [10]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_2,0,2011-01-30,11101,...,1,2011,None,None,None,None,0,0,0,NaN
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_3,0,2011-01-31,11101,...,1,2011,None,None,None,None,0,0,0,NaN
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_4,0,2011-02-01,11101,...,2,2011,None,None,None,None,1,1,0,NaN
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_5,0,2011-02-02,11101,...,2,2011,None,None,None,None,1,0,1,NaN


In [11]:
# pre processing missing values of prices by transforming with mean price of that id
data['sell_price'].fillna(data.groupby('id')['sell_price'].transform('mean'),inplace=True)
data['d'] = data['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [12]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,8.285275
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_2,0,2011-01-30,11101,...,1,2011,None,None,None,None,0,0,0,8.285275
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_3,0,2011-01-31,11101,...,1,2011,None,None,None,None,0,0,0,8.285275
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_4,0,2011-02-01,11101,...,2,2011,None,None,None,None,1,1,0,8.285275
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,d_5,0,2011-02-02,11101,...,2,2011,None,None,None,None,1,0,1,8.285275


In [14]:
#creating lags features and downcasting it.
lags = [1,2,3,7,14,21,28]
for lag in tqdm(lags):
    data['lag_'+str(lag)] = data.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id',
                                          'state_id'],as_index=False)['sales'].shift(lag).astype(np.float16)
    
#creating rolling mean features and downcasting it.
window_size = [1,2,3,7,14,21,28, 60, 90]

for window in tqdm(window_size):
    data['rolling_mean'+str(window)] = data.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 
                                                     'state_id'])['sales'].transform(lambda x: x.rolling(window= window).mean())
    data['rolling_median'+str(window)] = data.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 
                                                     'state_id'])['sales'].transform(lambda x: x.rolling(window= window).median())
    data['rolling_std'+str(window)] = data.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 
                                                     'state_id'])['sales'].transform(lambda x: x.rolling(window= window).std())

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [1:37:32<00:00, 650.26s/it]


In [15]:
data['date'] = pd.to_datetime(data['date'])

data['day'] = data['date'].dt.day.astype(np.int8)
data['year'] = data['date'].dt.year.astype(np.int16)
data['week_no_in_month'] = data['day'].apply(lambda x: math.ceil(x/7)).astype(np.int8)
data['is_weekend'] = (data['wday']<=2).astype(np.int8)

In [18]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_std28,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,5,1
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,2,0,2011-01-30,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,5,1
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,5,0
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0


In [37]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_std28,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,5,1
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,2,0,2011-01-30,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,5,1
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,5,0
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,HOBBIES_1_001_CA_1_evaluation,5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0


In [40]:
category_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2','snap_CA','snap_TX','snap_WI']

for col in tqdm(category_cols):
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [35:43<00:00, 164.89s/it]


In [41]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_std28,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend
0,0,0,1,3,1437,14370,1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,5,1
1,0,0,1,3,1437,14370,2,0,2011-01-30,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,5,1
2,0,0,1,3,1437,14370,3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,5,0
3,0,0,1,3,1437,14370,4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0
4,0,0,1,3,1437,14370,5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 59 columns):
 #   Column            Dtype         
---  ------            -----         
 0   state_id          int32         
 1   store_id          int32         
 2   cat_id            int32         
 3   dept_id           int32         
 4   item_id           int32         
 5   id                int32         
 6   d                 int16         
 7   sales             int64         
 8   date              datetime64[ns]
 9   wm_yr_wk          int16         
 10  weekday           object        
 11  wday              int16         
 12  month             int16         
 13  year              int16         
 14  event_name_1      int32         
 15  event_type_1      int32         
 16  event_name_2      int32         
 17  event_type_2      int32         
 18  snap_CA           int32         
 19  snap_TX           int32         
 20  snap_WI           int32         
 21  sell_p

In [43]:
data['revenue'] = data['sales']*data['sell_price'].astype(np.float32)

In [44]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend,revenue
0,0,0,1,3,1437,14370,1,0,2011-01-29,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,29,5,1,0.0
1,0,0,1,3,1437,14370,2,0,2011-01-30,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,30,5,1,0.0
2,0,0,1,3,1437,14370,3,0,2011-01-31,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,31,5,0,0.0
3,0,0,1,3,1437,14370,4,0,2011-02-01,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0.0
4,0,0,1,3,1437,14370,5,0,2011-02-02,11101,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,0.0


In [27]:
# data.info()

In [46]:
df= data[data['year']>= 2014].copy() 

In [47]:
df.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend,revenue
1068,0,0,1,3,1437,14370,1069,1,2014-01-01,11349,...,0.683333,1.0,0.812856,0.600000,0.0,0.761430,1,1,0,8.26
1069,0,0,1,3,1437,14370,1070,1,2014-01-02,11349,...,0.683333,1.0,0.812856,0.611111,0.0,0.759871,2,1,0,8.26
1070,0,0,1,3,1437,14370,1071,0,2014-01-03,11349,...,0.683333,1.0,0.812856,0.611111,0.0,0.759871,3,1,0,0.00
1071,0,0,1,3,1437,14370,1072,0,2014-01-04,11350,...,0.666667,0.5,0.816497,0.600000,0.0,0.761430,4,1,1,0.00
1072,0,0,1,3,1437,14370,1073,0,2014-01-05,11350,...,0.650000,0.0,0.819777,0.600000,0.0,0.761430,5,1,1,0.00


In [28]:
# df.info()

In [ ]:
for col in df.columns:
    if col.a

In [50]:
df= downcasting_datatype(df)

In [ ]:
df.to_pickle('df_from_2014_blog.pkl')

In [29]:
# df.info()

In [2]:
data= pd.read_pickle('df_from_2014_blog.pkl')

In [3]:
data.head()

,state_id,store_id,cat_id,dept_id,item_id,id,d,sales,date,wm_yr_wk,...,rolling_mean60,rolling_median60,rolling_std60,rolling_mean90,rolling_median90,rolling_std90,day_of_month,week_no_inmonth,is_weekend,revenue
1068,0,0,1,3,1437,14370,1069,1,2014-01-01,11349,...,0.683333,1.0,0.812856,0.600000,0.0,0.761430,1,1,0,8.26
1069,0,0,1,3,1437,14370,1070,1,2014-01-02,11349,...,0.683333,1.0,0.812856,0.611111,0.0,0.759871,2,1,0,8.26
1070,0,0,1,3,1437,14370,1071,0,2014-01-03,11349,...,0.683333,1.0,0.812856,0.611111,0.0,0.759871,3,1,0,0.00
1071,0,0,1,3,1437,14370,1072,0,2014-01-04,11350,...,0.666667,0.5,0.816497,0.600000,0.0,0.761430,4,1,1,0.00
1072,0,0,1,3,1437,14370,1073,0,2014-01-05,11350,...,0.650000,0.0,0.819777,0.600000,0.0,0.761430,5,1,1,0.00


In [4]:
data.columns

Index(['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_1', 'lag_2',
       'lag_3', 'lag_7', 'lag_14', 'lag_21', 'lag_28', 'rolling_mean1',
       'rolling_median1', 'rolling_std1', 'rolling_mean2', 'rolling_median2',
       'rolling_std2', 'rolling_mean3', 'rolling_median3', 'rolling_std3',
       'rolling_mean7', 'rolling_median7', 'rolling_std7', 'rolling_mean14',
       'rolling_median14', 'rolling_std14', 'rolling_mean21',
       'rolling_median21', 'rolling_std21', 'rolling_mean28',
       'rolling_median28', 'rolling_std28', 'rolling_mean60',
       'rolling_median60', 'rolling_std60', 'rolling_mean90',
       'rolling_median90', 'rolling_std90', 'day_of_month', 'week_no_inmonth',
       'is_weekend', 'revenue'],
      dtype='object')

In [6]:
data.drop(['date','weekday','wm_yr_wk'],axis=1, inplace= True)

In [7]:
train= data[data['d'] <= 1885]
valid= data[(data['d'] >1885) & (data['d']<1914)]
test= data[(data['d']>= 1914) & (data['d'] <= 1941)]
evaluation= data[data['d'] > 1941]

X_train, y_train= train.drop('sales', axis= 1), train['sales']
X_valid, y_valid= valid.drop('sales', axis= 1), valid['sales']
X_test, y_test= test.drop('sales', axis= 1), test['sales']
X_eval, y_eval= evaluation.drop('sales', axis= 1), evaluation['sales']

print(X_train.shape,y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)
print(X_eval.shape, y_eval.shape)

(24910330, 56) (24910330,)
(853720, 56) (853720,)
(853720, 56) (853720,)
(853720, 56) (853720,)
